In [21]:
import time
import re
import json
import statistics as stats
import requests as req
import numpy as np
import pandas as pd
from unidecode import unidecode
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
import tqdm
import feedparser
import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [22]:
#driver configuration
opciones=Options()

opciones.add_experimental_option('excludeSwitches', ['enable-automation'])
opciones.add_experimental_option('useAutomationExtension', False)
opciones.headless=False    # si True, no aperece la ventana (headless=no visible)
opciones.add_argument('--start-maximized')         # comienza maximizado
#opciones.add_argument('user-data-dir=selenium')    # mantiene las cookies
#opciones.add_extension('driver_folder/adblock.crx')       # adblocker
opciones.add_argument('--incognito')

In [23]:
PATH = 'driver/chromedriver'

driver = webdriver.Chrome()   # abre una ventana de chrome 
                            

driver.get('https://www.google.es')

In [24]:
url = 'https://www.museodelprado.es/coleccion/obras-de-arte?'

In [25]:
driver.get(url)

# SCRAPEO URL FLORA

In [32]:
diccionario_flora = {}
pagina_actual = 1

while True:
    try:
        # Espera a que los elementos estén presentes
        wait = WebDriverWait(driver, 10)
        elementos = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')))
        
        nombre_lista = f'flora_{pagina_actual}'
        flora = []

        for elemento in elementos:
            try:
                href = elemento.get_attribute('href')
                flora.append(href)
            except StaleElementReferenceException:
                # Si se produce StaleElementReferenceException, intenta obtener el enlace nuevamente
                elemento = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')
                href = elemento.get_attribute('href')
                flora.append(href)

        # Almacenar la lista en el diccionario con el nombre correspondiente
        diccionario_flora[nombre_lista] = flora

        # Mostrar los resultados con print en lugar de return
        print(f'{nombre_lista}: {flora}')

        # Hacer clic en la flecha siguiente
        flecha_siguiente = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > a.flecha.mas.desk')
        flecha_siguiente.click()

        # Incrementar contador de página
        pagina_actual += 1

        # Esperar a que la nueva página se cargue completamente
        wait.until(EC.staleness_of(elemento))
    except TimeoutException:
        # Si no se encuentra la flecha de la página siguiente, salir del bucle
        break

flora_1: ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_547', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_365', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_389', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_424', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_394', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_390', 

In [42]:
diccionario_flora

{'flora_1': ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_547',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_365',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_389',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_424',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjectflora_394',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_flora@@@pm:floraNode=http://museodelprado.es/items/biologicalobjec

# SCRAPEO URL FAUNA

In [34]:
diccionario_fauna = {}
pagina_actual = 1

while True:
    try:
        # Espera a que los elementos estén presentes
        wait = WebDriverWait(driver, 10)
        elementos = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')))
        
        nombre_lista = f'fauna_{pagina_actual}'
        fauna = []

        for elemento in elementos:
            try:
                href = elemento.get_attribute('href')
                fauna.append(href)
            except StaleElementReferenceException:
                # Si se produce StaleElementReferenceException, intenta obtener el enlace nuevamente
                elemento = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')
                href = elemento.get_attribute('href')
                fauna.append(href)

        # Almacenar la lista en el diccionario con el nombre correspondiente
        diccionario_fauna[nombre_lista] = fauna

        # Mostrar los resultados con print en lugar de return
        print(f'{nombre_lista}: {fauna}')

        # Hacer clic en la flecha siguiente
        flecha_siguiente = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > a.flecha.mas.desk')
        flecha_siguiente.click()

        # Incrementar contador de página
        pagina_actual += 1

        # Esperar a que la nueva página se cargue completamente
        wait.until(EC.staleness_of(elemento))
    except TimeoutException:
        # Si no se encuentra la flecha de la página siguiente, salir del bucle
        break

fauna_1: ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_140', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_162', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_46', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_445', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_523', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://m

In [43]:
diccionario_fauna

{'fauna_1': ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_140',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_162',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_46',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_445',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLifeNode=http://museodelprado.es/items/biologicalobjectwildlife_523',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_wildlife@@@pm:wildLife

# SCRAPEO URL PERSONAJES

In [33]:
diccionario_personajes = {}
pagina_actual = 1

while True:
    try:
        # Espera a que los elementos estén presentes
        wait = WebDriverWait(driver, 10)
        elementos = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')))
        
        nombre_lista = f'personajes_{pagina_actual}'
        personajes = []

        for elemento in elementos:
            try:
                href = elemento.get_attribute('href')
                personajes.append(href)
            except StaleElementReferenceException:
                # Si se produce StaleElementReferenceException, intenta obtener el enlace nuevamente
                elemento = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')
                href = elemento.get_attribute('href')
                personajes.append(href)

        # Almacenar la lista en el diccionario con el nombre correspondiente
        diccionario_personajes[nombre_lista] = personajes

        # Mostrar los resultados con print en lugar de return
        print(f'{nombre_lista}: {personajes}')

        # Hacer clic en la flecha siguiente
        flecha_siguiente = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > a.flecha.mas.desk')
        flecha_siguiente.click()

        # Incrementar contador de página
        pagina_actual += 1

        # Esperar a que la nueva página se cargue completamente
        wait.until(EC.staleness_of(elemento))
    except TimeoutException:
        # Si no se encuentra la flecha de la página siguiente, salir del bucle
        break

personajes_1: ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=aar%C3%B3n', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abacuc', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abel', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abiga%C3%ADl', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abraham', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=absal%C3%B3n', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=acedo,%20diego%20de', 'https://www

In [44]:
diccionario_personajes

{'personajes_1': ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=aar%C3%B3n',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abacuc',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abel',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abiga%C3%ADl',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=abraham',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=absal%C3%B3n',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:p65_E36_p138_represents_person@@@ecidoc:p131_E82_p102_has_title=acedo,%20diego%20de

# SCRAPEO URL ESCUELAS

In [38]:
diccionario_escuelas = {}
pagina_actual = 1

while True:
    try:
        # Espera a que los elementos estén presentes
        wait = WebDriverWait(driver, 10)
        elementos = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')))
        
        nombre_lista = f'escuela_{pagina_actual}'
        escuela = []

        for elemento in elementos:
            try:
                href = elemento.get_attribute('href')
                escuela.append(href)
            except StaleElementReferenceException:
                # Si se produce StaleElementReferenceException, intenta obtener el enlace nuevamente
                elemento = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')
                href = elemento.get_attribute('href')
                escuela.append(href)

        # Almacenar la lista en el diccionario con el nombre correspondiente
        diccionario_escuelas[nombre_lista] = escuela

        # Mostrar los resultados con print en lugar de return
        print(f'{nombre_lista}: {escuela}')

        # Hacer clic en la flecha siguiente
        flecha_siguiente = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > a.flecha.mas.desk')
        flecha_siguiente.click()

        # Incrementar contador de página
        pagina_actual += 1

        # Esperar a que la nueva página se cargue completamente
        wait.until(EC.staleness_of(elemento))
    except TimeoutException:
        # Si no se encuentra la flecha de la página siguiente, salir del bucle
        break


escuela_1: ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_2', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_83', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_82', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_111', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_35', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_34', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_86', 'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_schoo

In [45]:
diccionario_escuelas

{'escuela_1': ['https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_2',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_83',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_82',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_111',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_35',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_34',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&ecidoc:has_school@@@pm:schoolNode=http://museodelprado.es/items/school_86',
  'https://www.museodelprado.es/coleccion/obras-de-arte?

In [39]:
diccionario_ubicacion = {}
pagina_actual = 1

while True:
    try:
        # Espera a que los elementos estén presentes
        wait = WebDriverWait(driver, 10)
        elementos = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')))
        
        nombre_lista = f'ubicacion_{pagina_actual}'
        ubicacion = []

        for elemento in elementos:
            try:
                href = elemento.get_attribute('href')
                ubicacion.append(href)
            except StaleElementReferenceException:
                # Si se produce StaleElementReferenceException, intenta obtener el enlace nuevamente
                elemento = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > ul > li > a')
                href = elemento.get_attribute('href')
                ubicacion.append(href)

        # Almacenar la lista en el diccionario con el nombre correspondiente
        diccionario_ubicacion[nombre_lista] = ubicacion

        # Mostrar los resultados con print en lugar de return
        print(f'{nombre_lista}: {ubicacion}')

        # Hacer clic en la flecha siguiente
        flecha_siguiente = driver.find_element(By.CSS_SELECTOR, '#modal-autor > div.indice-lista.no-letra > a.flecha.mas.desk')
        flecha_siguiente.click()

        # Incrementar contador de página
        pagina_actual += 1

        # Esperar a que la nueva página se cargue completamente
        wait.until(EC.staleness_of(elemento))
    except TimeoutException:
        # Si no se encuentra la flecha de la página siguiente, salir del bucle
        break


ubicacion_1: ['https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=%C3%81bside', 'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Ayuntamiento%20de%20Albacete', 'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Diputaci%C3%B3n%20Provincial%20de%20Albacete', 'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Subdelegaci%C3%B3n%20del%20Gobierno%20en%20Albacete', 'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Tribunal%20Superior%20de%20Justicia%20de%20Castilla-La%20Mancha', 'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Alcal%C3%A1%20de%20Henares%20-%20Ayuntamiento%20de%20Alcal%C3%A1%20de%20Henares', 'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Alcal%C3%A1%20de%20Henares%20-

In [46]:
diccionario_ubicacion

{'ubicacion_1': ['https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=%C3%81bside',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Ayuntamiento%20de%20Albacete',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Diputaci%C3%B3n%20Provincial%20de%20Albacete',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Subdelegaci%C3%B3n%20del%20Gobierno%20en%20Albacete',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Albacete%20-%20Tribunal%20Superior%20de%20Justicia%20de%20Castilla-La%20Mancha',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Alcal%C3%A1%20de%20Henares%20-%20Ayuntamiento%20de%20Alcal%C3%A1%20de%20Henares',
  'https://www.museodelprado.es/coleccion/obras-de-arte?&cidoc:p55_has_current_location=Alcal%C3%A1%20d